In [1]:
import os
import random
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm

import torch
from torch.utils.data import Dataset

import sys
import evaluate
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoModelForCausalLM
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datetime import datetime
from sklearn.model_selection import train_test_split

SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cuda')

In [2]:
def log_message(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sys.stdout.write(f"[{timestamp}] {message}\n")
    sys.stdout.flush()

In [3]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, 'data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

# Model Load

In [4]:
model_name = 'yanolja/EEVE-Korean-Instruct-10.8B-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(DEVICE)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(40960, 4096)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm):

In [5]:
## 각각의 label별로 dataframe을 분할해줌
data_path = os.path.join(DATA_DIR, "Self_train/self_train_step4.csv")
origin_df = pd.read_csv(data_path)

df_list = []
for label in range(7) :
    label_df = origin_df[origin_df['target'] == label]
    df_list.append(label_df)
    print(label_df.head())



                     ID                         text  target
18  ynat-v1_train_00018  개와 함께한 정연 작가는 새로운 작품을 선보입니다       0
24  ynat-v1_train_00024  서울에 다시 오존주의보…도심·서북·동북권 발령종합       0
53  ynat-v1_train_00053             찍은 즐거운 개 카메라 앞에서       0
59  ynat-v1_train_00059      공룡 파충류보다는 타조·매 등 새에 가깝다       0
66  ynat-v1_train_00066     콩나물 면금, 물 나고 지 철에서 튀어 나와       0
                     ID                                            text  \
6   ynat-v1_train_00006                         프로야구 광주 롯데 경기, 비로 인해 취소   
7   ynat-v1_train_00007                아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...   
13  ynat-v1_train_00013                        아이 수리, 대 맥디 레의 대결에서 승리하다   
20  ynat-v1_train_00020  추신수, 타율 0.300 넘기며 시즌 마무리… 최지만, 홈런 2개 추가로 기록 경신   
23  ynat-v1_train_00023                아시안게임 목소리 높인 박항서 베트남이 일본 못 이길...   

    target  
6        1  
7        1  
13       1  
20       1  
23       1  
                     ID                            text  target
2   ynat-v1_train_00002       김정은, 자주통일 새

In [7]:
log_message(f"Model: {model_name}")
log_message(f"Data length: {len(origin_df)}")

categories = [[] for i in range(7)]
categories

[2024-11-05 18:19:54] Model: yanolja/EEVE-Korean-Instruct-10.8B-v1.0
[2024-11-05 18:19:54] Data length: 4121


[[], [], [], [], [], [], []]

In [8]:
def generate_prompt(titles):
    prompt = f"""당신은 신문 기사 제목을 분석하여 공통된 주제를 추출하는 전문가입니다. 아래 제시된 신문 기사 제목들을 주의 깊게 읽고, 이들 사이에서 공통된 주제나 카테고리를 찾아내세요. 다음 지침을 따라주세요:

                    1. 각 제목을 주의 깊게 읽고 핵심 키워드와 주제를 파악합니다.
                    2. 제목들 사이의 공통점이나 연관성을 찾습니다.
                    3. 가장 적합한 공통 주제나 카테고리를 1-2개의 한국어 단어로 제시합니다.

                    주의사항:
                    - 모든 제목이 완벽하게 일치하는 주제를 가질 필요는 없습니다. 대다수의 제목을 아우르는 주제를 찾으세요.
                    - 너무 광범위하거나 모호한 주제는 피하고, 가능한 한 구체적인 주제를 제시하세요.
                    - 주제를 찾기 어려운 경우, 가장 많이 등장하는 키워드나 개념을 중심으로 주제를 도출하세요.

                    신문 기사 제목:
                    """
    for idx, title in enumerate(titles):
        prompt += f"{idx}. {title}\n"
    prompt += "\n공통카테고리: "
    return prompt

In [11]:
for label, df in enumerate(df_list) :
    for i in tqdm(range(0, len(df), 10), desc=f"Processing label {label}"):
        titles = df['text'].iloc[i:i+10].tolist()
        
        prompt = generate_prompt(titles)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.95,
                do_sample=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_text = generated_text.split("공통카테고리:")[-1].strip()
        print("Titles:")
        for title in titles:
            print(f"- {title}")
        print(f"Generated Category: {cleaned_text}\n")
        
        # 생성된 카테고리를 10개의 제목 각각에 대해 저장
        categories[label].append(cleaned_text)
        
with open("label_list.pkl","wb") as f:
    pickle.dump(categories, f)

log_message("Process completed successfully")

Processing label 0:   2%|▏         | 1/60 [00:01<01:28,  1.50s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Titles:
- 개와 함께한 정연 작가는 새로운 작품을 선보입니다
- 서울에 다시 오존주의보…도심·서북·동북권 발령종합
- 찍은 즐거운 개 카메라 앞에서
- 공룡 파충류보다는 타조·매 등 새에 가깝다
- 콩나물 면금, 물 나고 지 철에서 튀어 나와
- MBC 류현진 특집 다큐…MLB 전설에게 듣는 류현진
- 특별호, 금융조달의 새로운 신고서를 소개합니다
- 한산한 성산일출봉 주차장
- 온열 한증방에서 느껴보는 천방 스킨케어 효과
- 전 세계 개와 고양이들이 체험하는 연대감
Generated Category: "애완동물과 자연"



Processing label 0:   3%|▎         | 2/60 [00:02<01:09,  1.21s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Titles:
- 세아트 드라이빙 시뮬레이터
- 신간 도서, '끝내주는 법'을 공동체와 나누다
- 어촌으로 봄 여행 떠나볼까…해수부 체험 마을 20곳 추천
- 미세먼지 보통, 오후에 점차 개선 기대
- 모멘트 학교 급식U4칸막이lx곡
- 올해의 상, 멀리 짖는 개들
- 2021년 과학 기술 신년 계획 발표
- 2017 올해의연극 베스트3 오른 옥상 밭 고추는 왜 재공연
- 무등산에서 광주지역 올해 첫 얼음 관찰
- 호텔서 스타 셰프 음식 만난다…더플라자 레스토랑 4곳 오픈
Generated Category: 삶의 다양한 측면들



Processing label 0:   5%|▌         | 3/60 [00:05<01:51,  1.96s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Titles:
- 신간 답이 보이지 않는 상황을 견디는 힘
- 길 위의 조: 파를 외치다, 승리를 향해 외치다!
- 시판코아, 페스타이벌에서 멋진 공연 선보여
- 봄비 머금은 꽃망울
- 국립민속박물관 아카이브 만들기 특별전 개최
- 말과 여, 인류무형문화유산 해에 함께 나다.
- 그림자가 드리운 결함들: 픽-리 사건의 진실을 파헤치다
- 위안부 피해자 그림 선생님이 전하는 이야기
- 황금연휴 해, 즐거운 휴가 이용권 드립니다
- 춘천 MBC 나이야가라 한국방송대상 작품상 수상
Generated Category: - 문화 및 예술: 제목 1, 2, 5, 6, 7, 9는 예술, 문화 행사, 그리고 공연과 관련된 내용을 다루고 있습니다. 따라서 이 제목들을 아우르는 공통 카테고리는 문화 및 예술입니다.



Processing label 0:   8%|▊         | 5/60 [00:25<06:16,  6.84s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Titles:
- 부산국제코미디페스티벌에 참여합니다
- 연천 영하 3도, 북부 강 위로 풀려나다
- 드랙퀸 연극 조지 맥브라이드의 전설 내달 초연
- 게시판 제38회 한국출판학회상에 천재교육 등 선정
- 말 여행, 제주 맛집 나들이 대세! 생생한 계곡 축제 즐겨보세요
- 1등 빛, 경기서 초미세먼지 주의보 발령하다
- 아힘 프라이어가 소개하는 반지 4부작
- 등단 3년 차, 장남 인가나에게 지도를 주다
- 그 픽 톡 가 현실화율 세 삶의 터전
- 버려진 튜브로 만든 설치미술
Generated Category: 1. 공연 및 예술 행사
2. 기상 및 환경
3. 교육 및 출판
4. 관광 및 여행
5. 미디어 및 기술
6. 환경 및 지속 가능성
7. 사회 및 문화
8. 개인 발전 및 성장
9. 디자인 및 예술



Processing label 0:   8%|▊         | 5/60 [00:32<05:53,  6.42s/it]


KeyboardInterrupt: 